# Insérer titre pertinent et aguicheur 

In [ ]:
!pip install pandas
!pip install geopandas

On importe avant tout les librairies intéressantes !

In [5]:
import pandas as pd
import requests
import geopandas as gpd

On définit ensuite une fonction qui permet de récupérer des données à partir d'une API

In [6]:
def get_dataset_from_url(api_url):
    '''Imports a database from an API url.
    Args:
        api_url (string) : url of the wanted API
    Returns:
        pandas.DataFrame : dataframe
    '''
    req=requests.get(api_url)
    wb=req.json()
    df = pd.json_normalize(wb["results"])
    #dataset = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs = 4326)

    return df

## Téléchargement des données de consommation d'électricité annuelle par adresse

In [9]:
consumption_api_root="https://enedis.opendatasoft.com/api/explore/v2.1/catalog/datasets/consommation-annuelle-residentielle-par-adresse/records"

def get_consumption(api_root, year):
    consumption_api_url=f"{api_root}?refine=annee"+"%3A"+f"{year}"
    return get_dataset_from_url(consumption_api_url)

consumption_2018 = get_consumption(consumption_api_root, "2018")
consumption_2019 = get_consumption(consumption_api_root, "2019")
consumption_2020 = get_consumption(consumption_api_root, "2020")
consumption_2021 = get_consumption(consumption_api_root, "2021")

In [10]:
consumption_2018

,annee,code_iris,nom_iris,numero_de_voie,indice_de_repetition,type_de_voie,libelle_de_voie,code_commune,nom_commune,segment_de_client,nombre_de_logements,consommation_annuelle_totale_de_l_adresse_mwh,consommation_annuelle_moyenne_par_site_de_l_adresse_mwh,consommation_annuelle_moyenne_de_la_commune_mwh,adresse,tri_des_adresses,code_epci,code_departement,code_region
0,2018,593920202,Pont Allant,45,None,RUE,LEON BATHIAT,59392,MAUBEUGE,RESIDENTIEL,61,154.865,2.539,3.246,45 RUE LEON BATHIAT,580004,200043396,59,32
1,2018,492440101,Pommeraye,14,None,RUE,MARIE MOREAU,49244,MAUGES-SUR-LOIRE,RESIDENTIEL,21,107.920,5.139,6.274,14 RUE MARIE MOREAU,580013,200060010,49,52
2,2018,49244xxxx,Fictif - Mauges-sur-Loire,35,None,RUE,NATIONALE,49244,MAUGES-SUR-LOIRE,RESIDENTIEL,23,125.959,5.476,6.274,35 RUE NATIONALE,580014,200060010,49,52
3,2018,341540105,Carnon,135,None,RUE,DE LA GARDIOLE,34154,MAUGUIO,RESIDENTIEL,27,62.629,2.320,4.802,135 RUE DE LA GARDIOLE,580018,243400470,34,76
4,2018,341540102,Quartier ouest,54,None,RUE,DE LA MESANGE,34154,MAUGUIO,RESIDENTIEL,10,64.527,6.453,4.802,54 RUE DE LA MESANGE,580024,243400470,34,76
5,2018,341540105,Carnon,141,None,PLACE,DES ESQUIFS,34154,MAUGUIO,RESIDENTIEL,53,93.192,1.758,4.802,141 PLACE DES ESQUIFS,580029,243400470,34,76
6,2018,341540101,Centre ville,108,None,RUE,DU SAUT DU LOUP,34154,MAUGUIO,RESIDENTIEL,26,117.236,4.509,4.802,108 RUE DU SAUT DU LOUP,580040,243400470,34,76
7,2018,593930000,Maulde (commune non irisée),35,None,RUE,DE CHORETTE,59393,MAULDE,RESIDENTIEL,11,74.983,6.817,5.743,35 RUE DE CHORETTE,580067,200042190,59,32
8,2018,783800102,Quartier Bas,2,None,RUE,D AGNOU,78380,MAULE,RESIDENTIEL,12,55.399,4.617,6.907,2 RUE D AGNOU,580068,200034130,78,11
9,2018,783800102,Quartier Bas,7,None,RUE,DU PRESSOIR,78380,MAULE,RESIDENTIEL,13,79.217,6.094,6.907,7 RUE DU PRESSOIR,580074,200034130,78,11


In [11]:
consumption_2018['nom_iris'].unique()

array(['Pont Allant', 'Pommeraye', 'Fictif - Mauges-sur-Loire', 'Carnon',
       'Quartier ouest', 'Centre ville', 'Maulde (commune non irisée)',
       'Quartier Bas'], dtype=object)

Problème ! La base de données est pour l'instant trop petite !